In [46]:
import pandas as pd
from collections import Counter
import json
import pyodbc

In [54]:
conn = pyodbc.connect(driver='SQL Server;',
                      host='34.170.174.91;',
                      database='grupo7;',
                      uid='henry;',
                      pwd='CERtificado14',
                      TrustServerCertificate='yes;')
cursor = conn.cursor()

In [2]:
df = pd.read_pickle("./../Data/restaurants.pkl")

In [3]:
#Con esta función validamos si existe un elemento de lista dentro de la columna Tipo si es así lo devolvemos a texto.
df["Tipo"] = df["Tipo"].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [4]:
#Comprobamos la modificación
df

,Id_Restaurant,Nombre,Ciudad,Estado,Cod_postal,Latitud,Longitud,Tipo,Atributos
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,Los Angeles,CA,90005,34.058092,-118.29213,Korean restaurant,"{'Service options': ['Takeout', 'Dine-in', 'De..."
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,La Mirada,CA,90638,33.916402,-118.010855,Restaurant,"{'Service options': ['Outdoor seating', 'Curbs..."
2,0x87ec235c54d25b31:0x3b75fb5facc602f,"Sweet Rewards Gluten Free Bakery, LLC",Waukee,IA,50263,41.616079,-93.865487,"Bakery, Health food restaurant",{'Service options': ['Delivery']}
3,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,Austin,TX,78702,30.273985,-97.719564,Restaurant,"{'Service options': ['Delivery', 'Takeout', 'D..."
4,0x4cb29c1dea53f29f:0xc606af14a0995094,Benny's Famous Fried Clams,Portland,ME,04101,43.643366,-70.272457,Restaurant,"{'Service options': ['Delivery'], 'Amenities':..."
...,...,...,...,...,...,...,...,...,...
171414,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,Boise,PA,83702,43.61659,-116.202383,"Bars, Gastropubs, Sandwiches, Nightlife, Resta...","{'BikeParking': 'True', 'Alcohol': 'u'full_bar..."
171415,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,Boise,ID,83704,43.615401,-116.284689,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'WiFi': ''free'', 'RestaurantsGoodForGroups':..."
171416,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,Philadelphia,MO,19147,39.935982,-75.158665,"Restaurants, Mexican","{'WheelchairAccessible': 'False', 'Restaurants..."
171417,2O2K6SXPWv56amqxCECd4w,The Plum Pit,Aston,PA,19014,39.856185,-75.427725,"Restaurants, Comfort Food, Food, Food Trucks, ...","{'RestaurantsDelivery': 'False', 'BusinessAcce..."


In [5]:
df.shape

(171419, 9)

## Normalización del campo Tipo

Vamos a definir un diccionario para clasificar el tipo de restaurante, donde la clave será el tipo homologado y los valores harán referencia a las palabras que se van a buscar en los campos Nombre y Tipo para asociar el tipo de restaurante.

Para ello, vamos a hacer uso de los siguientes códigos para identificar las palabras que más se repiten.

In [6]:
# Separamos los elementos de la columna unida por comas o espacios, creando una lista de palabras
palabras = [palabra.strip() for texto in df['Tipo'] for palabra in texto.split(' ')]

# Contamos la frecuencia de cada palabra en la lista
frecuencia_palabras = Counter(palabras)

# Ordenamos las palabras por su frecuencia en orden descendente
palabras_mas_frecuentes = frecuencia_palabras.most_common()

print(palabras_mas_frecuentes)

[('restaurant,', 100515), ('restaurant', 53887), ('Restaurant', 49454), ('Restaurants,', 35211), ('Restaurant,', 29462), ('Pizza', 28641), ('American', 27539), ('shop,', 25205), ('Fast', 23439), ('&', 22088), ('food', 21926), ('Takeout', 20065), ('Food,', 18587), ('Breakfast', 14286), ('Restaurants', 12348), ('Sandwich', 12304), ('Bars,', 12042), ('Mexican', 11840), ('Delivery', 11375), ('Coffee', 10443), ('Caterer,', 10125), ('Chinese', 9657), ('Chicken', 9603), ('Italian', 7616), ('store,', 7550), ('Food', 7204), ('Nightlife,', 6956), ('Asian', 6656), ('shop', 6611), ('(Traditional),', 6068), ('Sandwiches,', 5651), ('Bar,', 5592), ('delivery,', 5383), ('Seafood', 5030), ('(New),', 4768), ('Hamburger', 4728), ('Event', 4648), ('Pizza,', 4493), ('Brunch,', 4486), ('Bar', 4360), ('Cafe,', 3834), ('Family', 3783), ('Sushi', 3758), ('Burgers,', 3698), ('Bakery,', 3377), ('wings', 3359), ('Services,', 3348), ('Wine', 3237), ('Ice', 3220), ('Convenience', 3157), ('Bars', 3154), ('Japanese',

In [7]:
# Separamos los elementos de la columna unida por comas o espacios, creando una lista de palabras
palabras = [palabra.strip() for texto in df['Nombre'] for palabra in texto.split(' ')]

# Contamos la frecuencia de cada palabra en la lista
frecuencia_palabras = Counter(palabras)

# Ordenamos las palabras por su frecuencia en orden descendente
palabras_mas_frecuentes = frecuencia_palabras.most_common()

print(palabras_mas_frecuentes)

[('&', 16217), ('Pizza', 16004), ('Restaurant', 14209), ('Grill', 8375), ('Cafe', 7540), ('The', 7364), ('Bar', 6009), ('Subway', 5792), ('and', 4774), ('Kitchen', 4052), ('House', 3837), ('Mexican', 3579), ('El', 2990), ('-', 2739), ('La', 2556), ('Pizzeria', 2389), ('Food', 2276), ('China', 2259), ('Sushi', 2066), ('Taco', 2017), ('Express', 1992), ('BBQ', 1974), ('Chicken', 1965), ('Deli', 1922), ('Cuisine', 1905), ('King', 1886), ('Chinese', 1762), ('Bistro', 1756), ("John's", 1746), ('Italian', 1740), ('of', 1699), ('Hut', 1699), ('Thai', 1676), ('Burger', 1645), ("Domino's", 1612), ('Taqueria', 1560), ('Papa', 1529), ('Market', 1440), ('Seafood', 1408), ('New', 1322), ('Little', 1308), ('Tacos', 1272), ("Dunkin'", 1266), ('Coffee', 1249), ('Garden', 1242), ("McDonald's", 1194), ('Bakery', 1187), ('Pub', 1169), ('Shop', 1121), ('Diner', 1105), ('Subs', 1034), ('Asian', 1026), ('Wings', 1017), ('Street', 1006), ('the', 955), ('Grille', 949), ('Lounge', 937), ('Tavern', 919), ('Buff

In [8]:
#Con la información anterior se definen las siguientes categorías.
catalogo = {
    "Comida Americana":["american","burg","hamburger","barbecue","bbq","grill","steak","sandwich","hot dog","brunch"],
    "Comida Italiana": ["ital","bistro","pizz","pasta"],
    "Comida Oriental":["korea","japan","sushi","thai","chin","asian","oriental","ramen","indian","dragon","dumpling"],
    "Comida Europea":["span","german","greek","french"],
    "Comida Latinoamericana":["mexic","taco","taque","dominican","braz","colomb","hondu","cuba","venez"],
    "Comida Rápida":["fast", "gas","mac's","hale pops","chicken","wings","subway","pizza hut","burguer king","domino's","papa john's","little caesars","mcdonald's","carl's"],
    "Saludable/Vegano":["health","vegan","fit","salad","veget"],
    "Mariscos":["seafood","clams", "marisco","shell","fish"],
    "Reposteria/Pasteleria/Dulcería": ["bake","donut","smoothie","ice","dessert","froz"],
    "Cafetería":["coffee","tea","cafe","lunch","lounge","Starbucks"],        
    "Bar":["bar","pub","drink"]
}

In [9]:
# Definimos la función para normalizar la columna tipo, en caso de que no se encuentre el valor del catálogo, se clasifica como "Otros".
def normalizar_tipo(restaurante):
    for key, value in catalogo.items():
        for v in value:
            if v in restaurante['Nombre'].lower() or v in restaurante["Tipo"].lower(): #Evaluamos si en las columnas está algún valor de las listas del catálogo
                return key
    return "Otros"

In [10]:
# Aplicamos la función a cada una de las filas del DF
df['tipo_homologado'] = df.apply(normalizar_tipo, axis=1)

In [11]:
# Visualizamos el cambio realizado
df

,Id_Restaurant,Nombre,Ciudad,Estado,Cod_postal,Latitud,Longitud,Tipo,Atributos,tipo_homologado
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,Los Angeles,CA,90005,34.058092,-118.29213,Korean restaurant,"{'Service options': ['Takeout', 'Dine-in', 'De...",Comida Oriental
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,La Mirada,CA,90638,33.916402,-118.010855,Restaurant,"{'Service options': ['Outdoor seating', 'Curbs...",Comida Rápida
2,0x87ec235c54d25b31:0x3b75fb5facc602f,"Sweet Rewards Gluten Free Bakery, LLC",Waukee,IA,50263,41.616079,-93.865487,"Bakery, Health food restaurant",{'Service options': ['Delivery']},Saludable/Vegano
3,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,Austin,TX,78702,30.273985,-97.719564,Restaurant,"{'Service options': ['Delivery', 'Takeout', 'D...",Otros
4,0x4cb29c1dea53f29f:0xc606af14a0995094,Benny's Famous Fried Clams,Portland,ME,04101,43.643366,-70.272457,Restaurant,"{'Service options': ['Delivery'], 'Amenities':...",Mariscos
...,...,...,...,...,...,...,...,...,...,...
171414,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,Boise,PA,83702,43.61659,-116.202383,"Bars, Gastropubs, Sandwiches, Nightlife, Resta...","{'BikeParking': 'True', 'Alcohol': 'u'full_bar...",Comida Americana
171415,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,Boise,ID,83704,43.615401,-116.284689,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'WiFi': ''free'', 'RestaurantsGoodForGroups':...",Reposteria/Pasteleria/Dulcería
171416,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,Philadelphia,MO,19147,39.935982,-75.158665,"Restaurants, Mexican","{'WheelchairAccessible': 'False', 'Restaurants...",Comida Latinoamericana
171417,2O2K6SXPWv56amqxCECd4w,The Plum Pit,Aston,PA,19014,39.856185,-75.427725,"Restaurants, Comfort Food, Food, Food Trucks, ...","{'RestaurantsDelivery': 'False', 'BusinessAcce...",Reposteria/Pasteleria/Dulcería


## Normalización de la columna Atributos

Vamos a hacer otra prueba con otro código para ver si mejora el rendimiento

In [12]:
df2 = df.copy()

In [13]:
# Crear una lista de todas las claves únicas de los diccionarios de la columna de atributos
keys = set()
for attrs in df2['Atributos']:
    if attrs:
        keys.update(attrs.keys())

In [14]:
for key in keys:
    df2 = df2.assign(**{key: [attrs.get(key) if attrs is not None else None for attrs in df2['Atributos']]})

In [16]:
df2.columns

Index(['Id_Restaurant', 'Nombre', 'Ciudad', 'Estado', 'Cod_postal', 'Latitud',
       'Longitud', 'Tipo', 'Atributos', 'tipo_homologado', 'From the business',
       'Alcohol', 'BusinessAcceptsBitcoin', 'RestaurantsPriceRange2',
       'Ambience', 'BikeParking', 'ByAppointmentOnly',
       'RestaurantsReservations', 'BusinessParking', 'Health and safety',
       'Atmosphere', 'Recycling', 'BYOB', 'RestaurantsAttire', 'Popular for',
       'BusinessAcceptsCreditCards', 'NoiseLevel', 'AgesAllowed', 'Highlights',
       'Caters', 'AcceptsInsurance', 'RestaurantsCounterService',
       'GoodForMeal', 'Dining options', 'RestaurantsDelivery', 'WiFi',
       'DriveThru', 'Planning', 'Health & safety', 'Crowd',
       'RestaurantsGoodForGroups', 'Accessibility', 'Smoking', 'DogsAllowed',
       'RestaurantsTakeOut', 'Corkage', 'Offerings', 'HasTV', 'BestNights',
       'Music', 'DietaryRestrictions', 'GoodForDancing', 'Amenities',
       'Service options', 'WheelchairAccessible', 'CoatCheck', 

In [17]:
df2

,Id_Restaurant,Nombre,Ciudad,Estado,Cod_postal,Latitud,Longitud,Tipo,Atributos,tipo_homologado,...,WheelchairAccessible,CoatCheck,BYOBCorkage,RestaurantsTableService,Open24Hours,Payments,HairSpecializesIn,OutdoorSeating,HappyHour,GoodForKids
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,Los Angeles,CA,90005,34.058092,-118.29213,Korean restaurant,"{'Service options': ['Takeout', 'Dine-in', 'De...",Comida Oriental,...,None,None,None,None,None,None,None,None,None,None
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,La Mirada,CA,90638,33.916402,-118.010855,Restaurant,"{'Service options': ['Outdoor seating', 'Curbs...",Comida Rápida,...,None,None,None,None,None,None,None,None,None,None
2,0x87ec235c54d25b31:0x3b75fb5facc602f,"Sweet Rewards Gluten Free Bakery, LLC",Waukee,IA,50263,41.616079,-93.865487,"Bakery, Health food restaurant",{'Service options': ['Delivery']},Saludable/Vegano,...,None,None,None,None,None,None,None,None,None,None
3,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,Austin,TX,78702,30.273985,-97.719564,Restaurant,"{'Service options': ['Delivery', 'Takeout', 'D...",Otros,...,None,None,None,None,None,None,None,None,None,None
4,0x4cb29c1dea53f29f:0xc606af14a0995094,Benny's Famous Fried Clams,Portland,ME,04101,43.643366,-70.272457,Restaurant,"{'Service options': ['Delivery'], 'Amenities':...",Mariscos,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171414,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,Boise,PA,83702,43.61659,-116.202383,"Bars, Gastropubs, Sandwiches, Nightlife, Resta...","{'BikeParking': 'True', 'Alcohol': 'u'full_bar...",Comida Americana,...,None,None,'no',None,None,None,None,None,None,False
171415,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,Boise,ID,83704,43.615401,-116.284689,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'WiFi': ''free'', 'RestaurantsGoodForGroups':...",Reposteria/Pasteleria/Dulcería,...,None,None,None,None,None,None,None,True,None,True
171416,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,Philadelphia,MO,19147,39.935982,-75.158665,"Restaurants, Mexican","{'WheelchairAccessible': 'False', 'Restaurants...",Comida Latinoamericana,...,False,None,None,True,None,None,None,False,False,True
171417,2O2K6SXPWv56amqxCECd4w,The Plum Pit,Aston,PA,19014,39.856185,-75.427725,"Restaurants, Comfort Food, Food, Food Trucks, ...","{'RestaurantsDelivery': 'False', 'BusinessAcce...",Reposteria/Pasteleria/Dulcería,...,True,None,None,None,None,None,None,None,None,None


In [18]:
#Revisamos los valores de las columnas agregadas
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171419 entries, 0 to 171418
Data columns (total 64 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Id_Restaurant               171419 non-null  object
 1   Nombre                      171419 non-null  object
 2   Ciudad                      171419 non-null  object
 3   Estado                      171419 non-null  object
 4   Cod_postal                  171419 non-null  object
 5   Latitud                     171419 non-null  object
 6   Longitud                    171419 non-null  object
 7   Tipo                        171419 non-null  object
 8   Atributos                   170436 non-null  object
 9   tipo_homologado             171419 non-null  object
 10  From the business           4756 non-null    object
 11  Alcohol                     36757 non-null   object
 12  BusinessAcceptsBitcoin      6226 non-null    object
 13  RestaurantsPriceRange2      4

Revisamos por rangos de la cantidad de registros de las nuevas columnas con los atributos para dimensionar la cantidad que hay

In [20]:
for row in df2.columns:
    if df2[row].count() < 20000:
        print (f"la columna {row} tiene: ", df2[row].count())

la columna From the business tiene:  4756
la columna BusinessAcceptsBitcoin tiene:  6226
la columna ByAppointmentOnly tiene:  3116
la columna Health and safety tiene:  6151
la columna Recycling tiene:  1
la columna BYOB tiene:  3893
la columna AgesAllowed tiene:  77
la columna AcceptsInsurance tiene:  19
la columna RestaurantsCounterService tiene:  18
la columna DriveThru tiene:  5338
la columna Smoking tiene:  3246
la columna DogsAllowed tiene:  10702
la columna Corkage tiene:  3180
la columna BestNights tiene:  4274
la columna Music tiene:  5284
la columna DietaryRestrictions tiene:  27
la columna GoodForDancing tiene:  3418
la columna WheelchairAccessible tiene:  12514
la columna CoatCheck tiene:  4001
la columna BYOBCorkage tiene:  1377
la columna RestaurantsTableService tiene:  18192
la columna Open24Hours tiene:  20
la columna HairSpecializesIn tiene:  2
la columna HappyHour tiene:  12169


In [21]:
for row in df2.columns:
    if df2[row].count() >= 20000 and df2[row].count() < 40000:
        print (f"la columna {row} tiene: ", df2[row].count())

la columna Alcohol tiene:  36757
la columna Ambience tiene:  38011
la columna BikeParking tiene:  32337
la columna RestaurantsReservations tiene:  39342
la columna RestaurantsAttire tiene:  35770
la columna NoiseLevel tiene:  32185
la columna Caters tiene:  31542
la columna GoodForMeal tiene:  26509
la columna WiFi tiene:  34527
la columna Planning tiene:  23621
la columna Health & safety tiene:  37398
la columna RestaurantsGoodForGroups tiene:  37930
la columna HasTV tiene:  38329
la columna OutdoorSeating tiene:  39935
la columna GoodForKids tiene:  37518


In [22]:
for row in df2.columns:
    if df2[row].count() >= 40000 and df2[row].count() < 100000:
        print (f"la columna {row} tiene: ", df2[row].count())

la columna RestaurantsPriceRange2 tiene:  40533
la columna BusinessParking tiene:  41691
la columna Atmosphere tiene:  84396
la columna Popular for tiene:  81183
la columna BusinessAcceptsCreditCards tiene:  41356
la columna Highlights tiene:  45272
la columna Dining options tiene:  74510
la columna RestaurantsDelivery tiene:  43158
la columna Crowd tiene:  71871
la columna Accessibility tiene:  90542
la columna RestaurantsTakeOut tiene:  44166
la columna Offerings tiene:  97181
la columna Payments tiene:  54444


In [23]:
for row in df2.columns:
    if df2[row].count() >= 100000 and df2[row].count() < 170000:
        print (f"la columna {row} tiene: ", df2[row].count())

la columna Amenities tiene:  100149
la columna Service options tiene:  120107


In [24]:
#Contamos cómo aparecen los valores sin ninguna modificación (solo es ejemplo de una) para ver cómo se comportan las columnas agregadas
df2["Smoking"].value_counts()

u'no'         1814
u'outdoor'    1257
u'yes'         153
'no'            12
'outdoor'        6
None             4
Name: Smoking, dtype: int64

Con el siguiente código se puede identificar las palabras que más se repiten por las columnas de Atributos

In [25]:
# Separamos los elementos de la columna unida por comas, creando una lista de palabras
palabras = [palabra.strip() for texto in df2['Smoking'] if texto is not None for palabra in texto.split(' ')]

# Contamos la frecuencia de cada palabra en la lista
frecuencia_palabras = Counter(palabras)

# Ordenamos las palabras por su frecuencia en orden descendente
palabras_mas_frecuentes = frecuencia_palabras.most_common()

print(palabras_mas_frecuentes)

[("u'no'", 1814), ("u'outdoor'", 1257), ("u'yes'", 153), ("'no'", 12), ("'outdoor'", 6), ('None', 4)]


Apartir del análisis anterior, podemos definir el diccionario que va a agrupar los atributos por atributos que se repiten o que son más generales, la clave será la nueva columna, mientras que los valores van a ser las columnas originales de donde se va a valorar si se cuenta con el atributo o no, en caso de que se encuentre el atributo la nueva columna tendrá el valor 1 (indica Verdadero o que sí cuenta con el atributo), y 0 si no se cuenta con ningún atributo.

In [26]:
#Realizamos una copia del DF
df3 = df2.copy()

In [27]:
dicc_completo = {
    "Ambiente": ["Ambience","GoodForDancing","Music","Atmosphere","NoiseLevel"], 
    "Para grupos": ["RestaurantsGoodForGroups","Crowd", "AgesAllowed"], 
    "Promociones": ["HappyHour","Offerings"], 
    "Estacionamiento":["BikeParking","BusinessParking"],    
    "Mejores noches": ["BestNights"],
    "Inclusivo":["WheelchairAccessible","DogsAllowed","Accessibility"], 
    "Amenidades":["HasTV","GoodForKids","WiFi","Amenities"],     
    "Exteriores":["OutdoorSeating"],
    "Alcohol":["BYOB","Corkage","BYOBCorkage","Alcohol"], 
    "Restricciones en dieta":["DietaryRestrictions"],
    "Reservaciones":["ByAppointmentOnly","RestaurantsReservations","Planning"],    
    "Tipo de servicio":["DriveThru","RestaurantsDelivery","RestaurantsTakeOut","RestaurantsTableService","RestaurantsCounterService","Caters","Service options"],    #  
    "Guardaropa":["CoatCheck"],    
    "24 horas":["Open24Hours"],
    "Tipos de pago":["BusinessAcceptsCreditCards","BusinessAcceptsBitcoin","Payments"], 
    "Opciones (Des,Com,Cena)":["Dining options","GoodForMeal","Popular for"], 
    "Tipo director":["From the business"],
    "Vestimenta":["RestaurantsAttire"],
    "Salud y Seguridad": ["Health and safety","Health & safety"],    
    "Rango de precios":["RestaurantsPriceRange2"],
    "Aspectos destacados":["Highlights"], 
    "Fumar":["Smoking"],
}

La siguiente línea nos va a permitir crear las nuevas columnas a partir del diccionario definido.

In [28]:
# iterar sobre las columnas del diccionario de evaluación
for nueva_columna, columnas_evaluacion in dicc_completo.items():
    # seleccionar las columnas a evaluar del DataFrame original
    columnas_a_evaluar = [df3[columna] for columna in columnas_evaluacion]
    # evaluar si todas las columnas contienen valores nulos
    evaluacion = pd.concat(columnas_a_evaluar, axis=1).isna().all(axis=1)
    # establecer los valores de la nueva columna como 1 o 0 según el resultado de la evaluación
    nueva_columna_valor = evaluacion.apply(lambda x: 0 if x else 1)
    # agregar la nueva columna al DataFrame de resultados
    df3[nueva_columna] = nueva_columna_valor

In [29]:
df3

,Id_Restaurant,Nombre,Ciudad,Estado,Cod_postal,Latitud,Longitud,Tipo,Atributos,tipo_homologado,...,Guardaropa,24 horas,Tipos de pago,"Opciones (Des,Com,Cena)",Tipo director,Vestimenta,Salud y Seguridad,Rango de precios,Aspectos destacados,Fumar
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,Los Angeles,CA,90005,34.058092,-118.29213,Korean restaurant,"{'Service options': ['Takeout', 'Dine-in', 'De...",Comida Oriental,...,0,0,0,0,0,0,0,0,0,0
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,La Mirada,CA,90638,33.916402,-118.010855,Restaurant,"{'Service options': ['Outdoor seating', 'Curbs...",Comida Rápida,...,0,0,0,1,0,0,1,0,0,0
2,0x87ec235c54d25b31:0x3b75fb5facc602f,"Sweet Rewards Gluten Free Bakery, LLC",Waukee,IA,50263,41.616079,-93.865487,"Bakery, Health food restaurant",{'Service options': ['Delivery']},Saludable/Vegano,...,0,0,0,0,0,0,0,0,0,0
3,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,Austin,TX,78702,30.273985,-97.719564,Restaurant,"{'Service options': ['Delivery', 'Takeout', 'D...",Otros,...,0,0,0,1,0,0,0,0,0,0
4,0x4cb29c1dea53f29f:0xc606af14a0995094,Benny's Famous Fried Clams,Portland,ME,04101,43.643366,-70.272457,Restaurant,"{'Service options': ['Delivery'], 'Amenities':...",Mariscos,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171414,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,Boise,PA,83702,43.61659,-116.202383,"Bars, Gastropubs, Sandwiches, Nightlife, Resta...","{'BikeParking': 'True', 'Alcohol': 'u'full_bar...",Comida Americana,...,0,0,1,1,0,1,0,1,0,0
171415,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,Boise,ID,83704,43.615401,-116.284689,"Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'WiFi': ''free'', 'RestaurantsGoodForGroups':...",Reposteria/Pasteleria/Dulcería,...,0,0,1,1,0,1,0,1,0,0
171416,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,Philadelphia,MO,19147,39.935982,-75.158665,"Restaurants, Mexican","{'WheelchairAccessible': 'False', 'Restaurants...",Comida Latinoamericana,...,0,0,1,1,0,1,0,1,0,0
171417,2O2K6SXPWv56amqxCECd4w,The Plum Pit,Aston,PA,19014,39.856185,-75.427725,"Restaurants, Comfort Food, Food, Food Trucks, ...","{'RestaurantsDelivery': 'False', 'BusinessAcce...",Reposteria/Pasteleria/Dulcería,...,0,0,1,0,0,0,0,1,0,0


In [30]:
df3["Fumar"].unique()

array([0, 1], dtype=int64)

Otra prueba con listas en lugar de diccionario

In [31]:
df3["Fumar"].value_counts()

0    168173
1      3246
Name: Fumar, dtype: int64

In [32]:
eliminar=[
"Ambience","GoodForDancing","Music","Atmosphere","NoiseLevel","RestaurantsGoodForGroups","Crowd", "AgesAllowed",
    "HappyHour","Offerings","BikeParking","BusinessParking","BestNights","WheelchairAccessible","DogsAllowed","Accessibility",
    "HasTV","GoodForKids","WiFi","Amenities","OutdoorSeating","BYOB","Corkage","BYOBCorkage","Alcohol","DietaryRestrictions",
    "ByAppointmentOnly","RestaurantsReservations","Planning","DriveThru","RestaurantsDelivery","RestaurantsTakeOut","RestaurantsTableService",
    "RestaurantsCounterService","Caters","Service options","CoatCheck","Open24Hours","BusinessAcceptsCreditCards","BusinessAcceptsBitcoin",
    "Payments","Dining options","GoodForMeal","Popular for","From the business","RestaurantsAttire","Health and safety","Health & safety",    
    "RestaurantsPriceRange2","Highlights","Smoking","HairSpecializesIn","AcceptsInsurance","Recycling","Smoking","Tipo","Atributos"
]

In [33]:
df4=df3.copy()

In [34]:
df4.columns

Index(['Id_Restaurant', 'Nombre', 'Ciudad', 'Estado', 'Cod_postal', 'Latitud',
       'Longitud', 'Tipo', 'Atributos', 'tipo_homologado', 'From the business',
       'Alcohol', 'BusinessAcceptsBitcoin', 'RestaurantsPriceRange2',
       'Ambience', 'BikeParking', 'ByAppointmentOnly',
       'RestaurantsReservations', 'BusinessParking', 'Health and safety',
       'Atmosphere', 'Recycling', 'BYOB', 'RestaurantsAttire', 'Popular for',
       'BusinessAcceptsCreditCards', 'NoiseLevel', 'AgesAllowed', 'Highlights',
       'Caters', 'AcceptsInsurance', 'RestaurantsCounterService',
       'GoodForMeal', 'Dining options', 'RestaurantsDelivery', 'WiFi',
       'DriveThru', 'Planning', 'Health & safety', 'Crowd',
       'RestaurantsGoodForGroups', 'Accessibility', 'Smoking', 'DogsAllowed',
       'RestaurantsTakeOut', 'Corkage', 'Offerings', 'HasTV', 'BestNights',
       'Music', 'DietaryRestrictions', 'GoodForDancing', 'Amenities',
       'Service options', 'WheelchairAccessible', 'CoatCheck', 

In [35]:
df4 = df4.drop(eliminar,axis=1)

In [36]:
#Confirmamos la manera en que quedaron las columnas
df4.columns

Index(['Id_Restaurant', 'Nombre', 'Ciudad', 'Estado', 'Cod_postal', 'Latitud',
       'Longitud', 'tipo_homologado', 'Ambiente', 'Para grupos', 'Promociones',
       'Estacionamiento', 'Mejores noches', 'Inclusivo', 'Amenidades',
       'Exteriores', 'Restricciones en dieta', 'Reservaciones',
       'Tipo de servicio', 'Guardaropa', '24 horas', 'Tipos de pago',
       'Opciones (Des,Com,Cena)', 'Tipo director', 'Vestimenta',
       'Salud y Seguridad', 'Rango de precios', 'Aspectos destacados',
       'Fumar'],
      dtype='object')

In [37]:
# Renombramos la columna "tipo_homologado"
df4=df4.rename(columns={"tipo_homologado":"Tipo"})

In [38]:
df4.columns

Index(['Id_Restaurant', 'Nombre', 'Ciudad', 'Estado', 'Cod_postal', 'Latitud',
       'Longitud', 'Tipo', 'Ambiente', 'Para grupos', 'Promociones',
       'Estacionamiento', 'Mejores noches', 'Inclusivo', 'Amenidades',
       'Exteriores', 'Restricciones en dieta', 'Reservaciones',
       'Tipo de servicio', 'Guardaropa', '24 horas', 'Tipos de pago',
       'Opciones (Des,Com,Cena)', 'Tipo director', 'Vestimenta',
       'Salud y Seguridad', 'Rango de precios', 'Aspectos destacados',
       'Fumar'],
      dtype='object')

In [39]:
df4.shape

(171419, 29)

Por último le vamos a agregar los valores de Rating Promedio y cantidad de reviews a cada restaurant. Para ello debemos importar las reviews.

In [40]:
#Importamos datasets de reviews
for i in range(1,10):
    if i == 1:
        reviews = pd.read_csv('./../Data/reviews_finales/reviews_1.csv',sep=';',escapechar='\\')
    else:
        reviews = pd.concat([reviews,pd.read_csv('./../Data/reviews_finales/reviews_'+str(i)+'.csv',sep=';',escapechar='\\')],ignore_index=True)
reviews.head()

C:\Users\isaac avelino\AppData\Local\Temp\ipykernel_18696\94576222.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.concat([reviews,pd.read_csv('./../Data/reviews_finales/reviews_'+str(i)+'.csv',sep=';',escapechar='\\')],ignore_index=True)


,Id_Usuario,Id_Restaurant,Rating,Reseña,Timestamp
0,117975922625527873536.0,0x8889221157fb3455:0x5c125c40c3eccc2a,4,"On the higher end of price for pizza, but they...",1463443013514
1,114316524790614294528.0,0x8889221157fb3455:0x5c125c40c3eccc2a,3,"Food was ok, felt like the atmosphere as well ...",1447623939865
2,113632573920508362752.0,0x8889221157fb3455:0x5c125c40c3eccc2a,4,"Good food, service so so",1469293549247
3,107684037632376815616.0,0x8889221157fb3455:0x5c125c40c3eccc2a,5,Love it,1442279219480
4,112603593315264724992.0,0x8889221157fb3455:0x5c125c40c3eccc2a,5,Yum,1382634896130


In [41]:
#Calculamos el rating promedio y la cantidad de reviews
rating_promedio = reviews.groupby(by='Id_Restaurant').mean().drop(columns='Timestamp')
rating_promedio['Cant_reviews'] = reviews.groupby(by='Id_Restaurant').count()[['Rating']]['Rating'].values
rating_promedio.rename(columns={'Rating':'Rating_promedio'},inplace=True)
rating_promedio.reset_index(inplace=True)
rating_promedio.head()

C:\Users\isaac avelino\AppData\Local\Temp\ipykernel_18696\675123256.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  rating_promedio = reviews.groupby(by='Id_Restaurant').mean().drop(columns='Timestamp')


,Id_Restaurant,Rating_promedio,Cant_reviews
0,---kPU91CF4Lq2-WlRu9Lw,4.500000,24
1,--0iUa4sNDFiZFrAdIWhZQ,3.214286,14
2,--7PUidqRWpRSpXebiyxTg,1.750000,12
3,--8IbOsAAxjKRoYsBFL-PA,2.925926,27
4,--ZVrH2X2QXBFdCilbirsw,4.722222,36


In [56]:
#Mergeamos con df4
merge = pd.merge(df4,rating_promedio,on='Id_Restaurant')
merge.head()

,Id_Restaurant,Nombre,Ciudad,Estado,Cod_postal,Latitud,Longitud,Tipo,Ambiente,Para grupos,...,Tipos de pago,"Opciones (Des,Com,Cena)",Tipo director,Vestimenta,Salud y Seguridad,Rango de precios,Aspectos destacados,Fumar,Rating_promedio,Cant_reviews
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,Los Angeles,CA,90005,34.058092,-118.29213,Comida Oriental,1,0,...,0,0,0,0,0,0,0,0,4.428571,14
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,La Mirada,CA,90638,33.916402,-118.010855,Comida Rápida,1,1,...,0,1,0,0,1,0,0,0,4.833333,6
2,0x87ec235c54d25b31:0x3b75fb5facc602f,"Sweet Rewards Gluten Free Bakery, LLC",Waukee,IA,50263,41.616079,-93.865487,Saludable/Vegano,0,0,...,0,0,0,0,0,0,0,0,5.000000,9
3,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,Austin,TX,78702,30.273985,-97.719564,Otros,1,1,...,0,1,0,0,0,0,0,0,4.125000,8
4,0x4cb29c1dea53f29f:0xc606af14a0995094,Benny's Famous Fried Clams,Portland,ME,04101,43.643366,-70.272457,Mariscos,0,0,...,0,0,0,0,0,0,0,0,2.833333,6


In [57]:
#Vemos que la cantidad de registros coincida
merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171419 entries, 0 to 171418
Data columns (total 31 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Id_Restaurant            171419 non-null  object 
 1   Nombre                   171419 non-null  object 
 2   Ciudad                   171419 non-null  object 
 3   Estado                   171419 non-null  object 
 4   Cod_postal               171419 non-null  object 
 5   Latitud                  171419 non-null  object 
 6   Longitud                 171419 non-null  object 
 7   Tipo                     171419 non-null  object 
 8   Ambiente                 171419 non-null  int64  
 9   Para grupos              171419 non-null  int64  
 10  Promociones              171419 non-null  int64  
 11  Estacionamiento          171419 non-null  int64  
 12  Mejores noches           171419 non-null  int64  
 13  Inclusivo                171419 non-null  int64  
 14  Amen

In [67]:
merge.rename(columns={"Id_Restaurant":'Id_Restaurante', "Para grupos":'Para_grupos', 'Mejores noches':' Mejores_noches', 'Restricciones en dieta': 'Restricciones_en_dieta', 
                      'Tipo de servicio':'Tipo_de_servicio', '24 horas':'Horas', 'Tipos de pago':'Tipos_de_pago', 'Tipo director': 'Tipo_director', 'Salud y Seguridad': 'Salud_y_Seguridad',
                         'Rango de precios':'Rango_de_precios', 'Aspectos destacados': 'Aspectos_destacados', 'Opciones (Des,Com,Cena)':'Opciones'}, inplace = True)

In [68]:
merge.info()

,Id_Restaurante,Nombre,Ciudad,Estado,Cod_postal,Latitud,Longitud,Tipo,Ambiente,Para_grupos,...,Tipos_de_pago,Opciones,Tipo_director,Vestimenta,Salud_y_Seguridad,Rango_de_precios,Aspectos_destacados,Fumar,Rating_promedio,Cant_reviews
0,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,Los Angeles,CA,90005,34.058092,-118.29213,Comida Oriental,1,0,...,0,0,0,0,0,0,0,0,4.428571,14
1,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,La Mirada,CA,90638,33.916402,-118.010855,Comida Rápida,1,1,...,0,1,0,0,1,0,0,0,4.833333,6


In [63]:
cursor.execute('''
 CREATE TABLE dbo.Restaurantes_Homologados (
        Id_Restaurante nvarchar(200) not null,
        Nombre nvarchar(max) not null,	
        Ciudad nvarchar(max) not null,	
        Estado nvarchar(max) not null,
        Cod_postal nvarchar(max) not null,	
        Latitud	nvarchar(max) not null,
        Longitud nvarchar(max) not null,	
        Tipo nvarchar(max) not null,	
        Ambiente nvarchar(max) not null,	
        Para_grupos nvarchar(max) not null,	
        Promociones nvarchar(max) not null,	
        Estacionamiento	nvarchar(max) not null,
        Mejores_noches nvarchar(max) not null,
        Inclusivo nvarchar(max) not null,	
        Amenidades nvarchar(max) not null,	
        Exteriores nvarchar(max) not null,	
        Restricciones_en_dieta	nvarchar(max) not null,
        Reservaciones nvarchar(max) not null,
        Tipo_de_servicio nvarchar(max) not null,	
        Guardaropa nvarchar(max) not null,
        Horas nvarchar(max) not null,	
        Tipos_de_pago nvarchar(max) not null,
        Opciones nvarchar(max) not null,
        Tipo_director nvarchar(max) not null, 
        Vestimenta nvarchar(max) not null, 
        Salud_y_Seguridad nvarchar(max) not null, 
        Rango_de_precios nvarchar(max) not null,
        Aspectos_destacados nvarchar(max) not null,	
        Fumar nvarchar(max) not null,	
        Rating_promedio nvarchar(max) not null,	
        Cant_reviews nvarchar(max) not null
);
               ''')

In [ ]:
for index, row in merge.iterrows():
        cursor.execute("INSERT INTO dbo.Restaurantes_Homologados (Id_Restaurante, Nombre, Ciudad, Estado, Cod_postal, Latitud, Longitud, Tipo, Ambiente, Para_grupos, Promociones, Estacionamiento, Mejores_noches, Inclusivo,	Amenidades, Exteriores,	Restricciones_en_dieta,	Reservaciones, Tipo_de_servicio, Guardaropa, Horas, Tipos_de_pago, Opciones, Tipo_director, Vestimenta,	Salud_y_Seguridad, Rango_de_precios, Aspectos_destacados, Fumar, Rating_promedio, Cant_reviews) values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                  row.Id_Restaurante, row.Nombre, row.Ciudad, row.Estado, row.Cod_postal, row.Latitud, row.Longitud, row.Tipo, row.Ambiente, row.Para_grupos, row.Promociones, row.Estacionamiento, row.Mejores_noches, row.Inclusivo,	row.Amenidades, row.Exteriores,	row.Restricciones_en_dieta, row.Reservaciones, row.Tipo_de_servicio, row.Guardaropa, row.Horas, row.Tipos_de_pago, row.Opciones, row.Tipo_director, row.Vestimenta, row.Salud_y_Seguridad, row.Rango_de_precios, row.Aspectos_destacados, row.Fumar, row.Rating_promedio, row.Cant_reviews
                   )
    
    
cursor.commit()

print("Termine de ingresar datosa")

In [64]:
#Exportamos csv
merge.to_csv("./../Data/restaurants_homolog.csv", index=False)